# Sensitivity Analysis Methods
#### **Andrew Attilio**
#### **2024.07.11**

These notes are synthesized from "Sensitvity analysis: A review of recent advances" by E. Borgonovo and E. Plischke. 

### Motivation

The authors provide some inspirational quotes from past research in sensitivity analysis, such as:

- “In order for the analysis to be useful it must provide information concerning the way in which our equilibrium quantities will change as a result of changes in the parameters."
- “The judicious application of sensitivity analysis techniques appears to be the key ingredient needed to draw out the maximum capabilities of mathematical modeling.”

### Our Goals / Potential Application

"A crucial step for a meaningful sensitivity analysis is the clear statement of the insights that we wish to obtain from the model."

Determine the effect of $\beta$, case counts, or other parameters on our likelihood function. This information can be used at each time step to alter our forecasts or change our model.

The gradient-based methods could also be utilized to send our perturbations in the correct direction to maximize likelihood.

### Harris Economic Order Quantity

Throught this notebook, we use the author's example model, the Harris Economic Order Quantity (EOQ):

$$y = \sqrt{\frac{240MS}{C}}$$

`y` is the optimal order quantity

`m` is the units per month

`s` is the setup cost of an order

`c` is the unit price of items in the stock


In [1]:
from dataclasses import dataclass
import jax.numpy as jnp


def eoq(m: float, c: float, s: float) -> float:
    """
    Economic order quantity function defined by Harris, 1990. 
    
    The number 240 is an "interest charge" per unit, per month, 
    corresponding to a yearly interest rate of 10 percent.
    
    :param x: dataclass representing a vector of model inputs. 
    :return: optimal order quantity
    """
    return jnp.sqrt(240 * m * s / c)

X = {'m': 100, 'c': 5, 's': 1}
y = eoq(X['m'], X['c'], X['s'])
print(y)

69.282036


## Local Sensitivity Methods
If we want to perform sensitivity analysis around some point of interest, 
then we are performing local analysis. This is in contrast to global 
or probabilistic analysis. 

These local methods include:
- One-at-a-Time
- Scenario Decomposition
- Partial Differentiation


## Sensitivity Analysis with Finite Differences
### One-at-a-Time (OAT) Methods

We assign a base case $x^0$ and a sensitivity case $x^+$ to the model 
inputs. We could also have an $x^-$ case, but that is not shown here.

For $x^+$, we increase each model input, one at a time, by 10% (this is an arbitrary choice and would change with the model). For an $x^-$ case, we could then **decrease** all inputs by 10%. 

Then, we compare the resulting $\Delta^+ y$ outputs (one for each changed parameter), where:

$$\Delta_i^+ y = g(x_i^+, x^0_{\neg i}) - y^0$$ 

So, in our example case, we might have:

$$\Delta_m^+ y = g(m^+, c^0, s^0) - y^0$$ 

In [2]:
# Initial parameters                                                                                        
x0 = {'m': 1230.0, 'c': 0.0135, 's': 2.15}                                                                    
y0 = eoq(x0['m'], x0['c'], x0['s'])                                                      
                                                                                                             
# Parameters with changes                                                                                   
x_plus = {'m': 1353, 'c': 0.01485, 's': 2.365}                                                              
                                                                                                             
# Calculate changes in output                                                                               
delta_y_m = eoq(x_plus['m'], x0['c'], x0['s']) - y0
delta_y_c = eoq(x0['m'], x_plus['c'], x0['s']) - y0
delta_y_s = eoq(x0['m'], x0['c'], x_plus['s']) - y0                
                                                                                                             
# Store deltas in a list                                                                                    
deltas = [delta_y_m, delta_y_c, delta_y_s]         
deltas = [delta.item() for delta in deltas]
                                                                                                             
# Print results                                                                                             
print("Base case:", y0)                                                                                     
print("Changes in model output with respect to m, c, and s:\n", deltas)     

                                                                                

Base case: 6856.627
Changes in model output with respect to m, c, and s:
 [334.6640625, -319.08984375, 334.6640625]


This indicates that when `m` is increased by 10%, the model output increases
 by 335 units. When `c` is increased, the model output decreases by 319 
 units, and so on. 
 

#### Question: 
For a Particle Filter, we can consider $\beta$ and case counts
 as inputs, with the likelihood as the output. Do we have a "best case" and "worst case"? Maybe a "maximum case" (high beta, high case count) and a "minimum case" (no new cases)?


In [3]:
delta_y_plus = eoq(x_plus['m'], x_plus['c'], x_plus['s']) - y0
sum_of_delta = jnp.sum(jnp.array(deltas))

print("Change in y from x0 to x_plus:", delta_y_plus)
print("Sum of individual changes:", sum_of_delta)

Change in y from x0 to x_plus: 334.66406
Sum of individual changes: 350.23828


**Note that these changes are not equal**. This is due to the fact that this method of 
sensitivity analysis does not account for interaction effects between the input variables.

## Visualization
The standard way to visualize these OAT changes is a **Tornado Diagram**. 

![tornado_diagram](./tornado_diagram.png)

## Scenario Decomposition
We start with a base case $x^0$, best case $x^+$, and worst case $x^-$.

**Problem**:
We could compare the outputs of $y(x^+)$, $y(x^0)$, $y(x^-)$. However, we would not know what caused the changes from one to another. The following methods attempt to rectify this issue.

We can decompose $\Delta y = g(x^+) - g(x^0)$ by utilizing finite differences, where:

$$\Delta y = \sum_{i=1}^{n} \Phi_i + \sum_{i<j} \Phi_{i,j} + ... + \Phi_{1,
2,...n}$$

And where:

\begin{align}
\Phi_i &= \Delta_i^+ y \\
\Phi_{i,j} &= \Delta_{i,j}^+ y - \Phi_i - \Phi_j \\
\Phi_{i,j,k} &= \Delta_{i,j,k}^+ y - \Phi_{i, j} \Phi_{i, k} - \Phi_{j,k} -
\Phi_i - \Phi_j - \Phi_k \\
...
\end{align}

So, $\Phi_i$ is just the one-at-a-time change in $y$ with respect to $x_i$. 
And $\Phi_{i, j}$ is an interaction effect from simultaneous change in $i$ and 
$j$.  


In [4]:
# Initial parameters                                                                                        
x0 = {'m': 1230.0, 'c': 0.0135, 's': 2.15}                                                                    
y0 = eoq(x0['m'], x0['c'], x0['s'])                                                      
                                                                                                             
# Parameters with changes                                                                                   
x_plus = {'m': 1353, 'c': 0.01485, 's': 2.365}   
 

In [5]:
# Calculate singleton phi                                    
# We are basically renaming earlier calculations here.
phi_m = eoq(x_plus['m'], x0['c'], x0['s']) - y0
phi_c = eoq(x0['m'], x_plus['c'], x0['s']) - y0
phi_s = eoq(x0['m'], x0['c'], x_plus['s']) - y0 
print("Phi m:", phi_m)
print("Phi c:", phi_c)
print("Phi s:", phi_s)

Phi m: 334.66406
Phi c: -319.08984
Phi s: 334.66406


In [6]:
# Second order phis
phi_mc = eoq(x_plus['m'], x_plus['c'], x0['s']) - y0 - phi_m - phi_c
phi_ms = eoq(x_plus['m'], x0['c'], x_plus['s']) - y0 - phi_m - phi_s
phi_cs = eoq(x_plus['m'], x_plus['c'], x0['s']) - y0 - phi_c - phi_s
print("Phi mc:", phi_mc)
print("Phi cs:", phi_cs)
print("Phi ms:", phi_ms)

Phi mc: -15.574219
Phi cs: -15.574219
Phi ms: 16.334473


In [7]:
# Third order phi
phi_mcs = (eoq(x_plus['m'], x_plus['c'], x_plus['s']) - y0 - phi_mc - phi_cs
           - phi_ms - phi_c - phi_m - phi_s)
print("Phi mcs:", phi_mcs)

Phi mcs: -0.7602539


Computational cost is $2^n$, which is not feasible as the number of inputs $n$ grows.

For a large number of inputs, the author recommends computing the first order ($\Phi_i$), total order ($\Phi_i^T$), and interaction ($\Phi_i^I$) sensitivity indices. This has a computational cost of $2n+2$. 

$$\Phi_i^T = \Phi_i + \sum_{j=1}^{n - 1}\Phi_{i,j} + ... + \Phi_{1,2,...,n}$$
$$\Phi_i^I = \Phi_i^T - \Phi_i$$

The interaction indicator $\Phi_i^I$ tells us how some input $x_i$ interacts with the 
remaining model inputs. 

## Differentiation-based Methods


In [8]:
import jax 
import jax.numpy as jnp

df_dm = jax.grad(eoq, argnums=0)(x0['m'], x0['c'], x0['s'])
print('Partial with respect to m:', df_dm)

Partial with respect to m: 2.7872467


In [9]:
df_dc = jax.grad(eoq, argnums=1)(x0['m'], x0['c'], x0['s'])
print('Partial with respect to c:', df_dc)

Partial with respect to c: -253949.12


In [10]:
df_ds = jax.grad(eoq, argnums=2)(x0['m'], x0['c'], x0['s'])
print('Partial with respect to s:', df_ds)

Partial with respect to s: 1594.5643


We might assume that `c` has by far the biggest effect on our output. 

However, the partial derivatives themselves are not comparable, because they are denominated in different units. Recall that `c` is a unit price, while `m` is units per month.

Thus, we define the Differential Importance Measure, $D_i$, which indicates the DIM of the variable $x_i$ in our input vector. 

The **elasticity** of $x_i$, denoted $E_i$, is an intermediate calculation. We multiply by $x_i^0$ to cancel out the difference in units between each $x_i$.

$$E_i = \frac{\partial g(x^0)}{\partial x_i} \frac{x_i^0}{g(x^0)}$$

$$D_i = \frac{E_i}{\sum_{j=1}^n E_j}$$

In [11]:
from typing import Callable

def elasticity(x: dict, func: Callable) -> jnp.ndarray:
    y0 = func(x['m'], x['c'], x['s'])
    inputs = x['m'], x['c'], x['s']
    elasticities = []
    for i, input in enumerate(inputs):
        partial = jax.grad(func, argnums=i)(x['m'], x['c'], x['s'])
        elasticities.append(partial * input / y0)
    return jnp.array(elasticities)

In [12]:
elastics = elasticity(x0, eoq)

In [13]:
from jax.typing import ArrayLike

def diff_importance(elasticities: ArrayLike) -> ArrayLike:
    return jnp.round(elasticities / jnp.sum(elasticities), 3)

In [14]:
diff_importance(elastics)

Array([ 1., -1.,  1.], dtype=float32)

We find that the Differential Importance Measures of each variable have the same magnitude, with $x_2$ having an opposite effect as the other two inputs.

Additivity is a convenient property of DIM. We see that if we change $x_1$ and $x_2$ at the same rate, the effect on the output is 0.  

**Partial derivatives and DIM do not account for interactions.** 

To account for interactions in differentiation-based sensitivity analysis, 
higher-order derivatives and 'joint reliability
 measures' could be used. 

### Other Methods -- Global Sensitivity
The above is a brief overview of local sensitivity methods. Probability-based global methods are outlined in the paper and could be a topic for further research. 